In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle
%matplotlib inline
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import accuracy_score, confusion_matrix,roc_curve,roc_auc_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report

In [ ]:
data=pd.read_csv('census_income.csv')
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.isna().sum()

In [ ]:
data.info()

There are few Object datatype features, so encoding is required

In [ ]:
print(data['Workclass'].value_counts().sort_values())

In [ ]:
print(data['Education'].value_counts().sort_values())

In [ ]:
print(data['Marital_status'].value_counts().sort_values())

In [ ]:
print(data['Occupation'].value_counts().sort_values())

In [ ]:
print(data['Relationship'].value_counts().sort_values())

In [ ]:
print(data['Race'].value_counts().sort_values())

In [ ]:
print(data['Sex'].value_counts().sort_values())

In [ ]:
print(data['Native_country'].value_counts().sort_values())

In [ ]:
print(data['Income'].value_counts().sort_values())

Some features have "?" in their data. Based on dataset, it would be harful to use imputation to replace ?, so before encoding, we must remove the lines for Occupation and Workclass.
For "?" in Country, we can replace it with other and encode it

In [ ]:
from sklearn.preprocessing import LabelEncoder
lab_enc=LabelEncoder()

In [ ]:
df=lab_enc.fit_transform(data['Workclass'])
pd.Series(df)
data['Workclass']=df
df=lab_enc.fit_transform(data['Education'])
pd.Series(df)
data['Education']=df
df=lab_enc.fit_transform(data['Marital_status'])
pd.Series(df)
data['Marital_status']=df
df=lab_enc.fit_transform(data['Occupation'])
pd.Series(df)
data['Occupation']=df
df=lab_enc.fit_transform(data['Relationship'])
pd.Series(df)
data['Relationship']=df
df=lab_enc.fit_transform(data['Race'])
pd.Series(df)
data['Race']=df
df=lab_enc.fit_transform(data['Sex'])
pd.Series(df)
data['Sex']=df
df=lab_enc.fit_transform(data['Native_country'])
pd.Series(df)
data['Native_country']=df
df=lab_enc.fit_transform(data['Income'])
pd.Series(df)
data['Income']=df

In [ ]:
data.describe()

In [ ]:
data

In [ ]:
plt.figure(figsize=(25,25),facecolor='white')
p=1
for column in data:
    if p <=15:
        ax=plt.subplot(3,5,p)
        sns.distplot(data[column])
        plt.xlabel(column,fontsize=20)
    p+=1
plt.tight_layout()

In [ ]:
y=data['Income']
x=data.drop(columns=['Income'])

In [ ]:
plt.figure(figsize=(25,25),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=15:
        ax=plt.subplot(4,4,plotnumber)
        plt.scatter(x[column],y)
        plt.xlabel(column,fontsize=20)
        plt.ylabel('Income',fontsize=20)
    plotnumber+=1
plt.tight_layout()

In [ ]:
data_corr=data.corr().abs()
plt.figure(figsize=(20,20))
sns.heatmap(data_corr,annot=True,annot_kws={'size':10})
plt.show

Since Education is already taken under Education_Num, and has weaker relation with income, so it can be exluded

In [ ]:
from sklearn.feature_selection import SelectKBest,f_classif

In [ ]:
best_features=SelectKBest(score_func=f_classif,k='all')

fit=best_features.fit(x,y)

data_scores=pd.DataFrame(fit.scores_)

data_columns=pd.DataFrame(x.columns)

feature_scores=pd.concat([data_columns,data_scores],axis=1)

feature_scores.columns=['Feature_Name','Score']

print(feature_scores.nlargest(13,'Score'))

In [ ]:
plt.figure(figsize=(15,15),facecolor='white')
plotnumber=1
for column in x:
    if plotnumber<=16:
        ax=plt.subplot(4,4,plotnumber)
        sns.lineplot(x='Income',y=column,data=data)
        plt.xlabel('Income',fontsize=10)
        plt.ylabel(column,fontsize=10)
    plotnumber+=1
plt.tight_layout()

In [ ]:
plt.figure(figsize=(15,15))
plotno=1
for column in x:
    if plotno<=16:
        ax=plt.subplot(4,4,plotno)
        sns.stripplot(y,x[column])
    plotno+=1
plt.show()

Based on heatmap, selectk best feature, and lineplot we can determine which features have best relation with Attrition

In [ ]:
y=data['Income']
x=data[['Education_num','Relationship','Age','Hours_per_week','Capital_gain','Sex','Marital_status','Capital_loss','Occupation','Race','Workclass','Native_country']]

In [ ]:
plt.figure(figsize=(20,20),facecolor='red')
plotno=1
for column in x:
    if plotno<=14:
        ax=plt.subplot(3,4,plotno)
        sns.boxplot(data=x[column])
        plt.xlabel(column,fontsize=12)
    plotno+=1
plt.show()

Outliers are present for many features, but they can be understood based om the features type

In [ ]:
scaler=StandardScaler()
x_scaled=scaler.fit_transform(x)
x_train,x_test,y_train,y_test=train_test_split(x_scaled,y,test_size=0.17,random_state=333)

In [ ]:
def metric_score(clf,x_train,x_test,y_train,y_test,train=True):
    if train:
        y_pred=clf.predict(x_train)
        print('\n----Train result----')
        print(f'Accuracy Score:{accuracy_score(y_train,y_pred)*100:.2f}%')
        
    elif train==False:
        pred=clf.predict(x_test)
        print('\n----Test result----')
        print(f'Accuracy Score:{accuracy_score(y_test,pred)*100:.2f}%')

        print('\n\n Test Classification report \n',classification_report(y_test,pred,digits=2))

In [ ]:
knn=KNeighborsClassifier()
knn.fit(x_train,y_train)

In [ ]:
metric_score(knn,x_train,x_test,y_train,y_test,train=True)
metric_score(knn,x_train,x_test,y_train,y_test,train=False)

In [ ]:
from sklearn.linear_model import LogisticRegression
log_reg=LogisticRegression()
log_reg.fit(x_train,y_train)

In [ ]:
metric_score(log_reg,x_train,x_test,y_train,y_test,train=True)
metric_score(log_reg,x_train,x_test,y_train,y_test,train=False)

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
bag_knn=BaggingClassifier(KNeighborsClassifier(n_neighbors=9),
                          n_estimators=6,max_samples=0.75,
                          bootstrap=True,random_state=366,oob_score=True)

In [ ]:
bag_knn.fit(x_train,y_train)

In [ ]:
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=True)
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rnd_clf=RandomForestClassifier()
rnd_clf.fit(x_train,y_train)

In [ ]:
metric_score(rnd_clf,x_train,x_test,y_train,y_test,train=True)
metric_score(rnd_clf,x_train,x_test,y_train,y_test,train=False)

Based on the accuracy scores of several models, Bag KNN has closest Train and Test result, the difference in bagknn model's train and test is smallest

So we will move forward with Bagging - KNN model

In [ ]:
from sklearn.model_selection import KFold,cross_val_score

In [ ]:
cross_val_score(bag_knn,x_scaled,y,cv=5)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid = {
    'base_estimator__n_neighbors' : [1, 2, 3, 4, 5],
    'max_samples' : [0.05, 0.1, 0.2, 0.5]}

grid_search = GridSearchCV(BaggingClassifier(KNeighborsClassifier(),n_estimators = 9, max_features = 12),param_grid)
grid_search.fit(x_train, y_train)

In [ ]:
best_parameters=grid_search.best_params_
print(best_parameters)

In [ ]:
bag_knn=BaggingClassifier(KNeighborsClassifier({'base_estimator__n_neighbors': 5, 'max_samples': 0.2}))

In [ ]:
bag_knn.fit(x_train,y_train)

In [ ]:
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=True)
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=False)

Score did not improve so we'll use original Bagging KNNeighbors model

In [ ]:
bag_knn=BaggingClassifier(KNeighborsClassifier(n_neighbors=9),
                          n_estimators=6,max_samples=0.75,
                          bootstrap=True,random_state=366,oob_score=True)

bag_knn.fit(x_train,y_train)

In [ ]:
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=True)
metric_score(bag_knn,x_train,x_test,y_train,y_test,train=False)

In [ ]:
pickle.dump(bag_knn,open('Census Income','wb'))